In [2]:
initialize_resource_paths <- function() {
  # Google Colab에서는 R에서 직접 마운트할 수 없으므로 생략합니다.
  # Google Colab에서는 별도의 마운트 과정이 필요하지 않습니다.

  # root 디렉토리 경로
  root_directory <- '/content/drive/MyDrive/Dataset/'

  # 데이터셋 경로 설정
  output <- list()
  output$dataset1 <- paste0(root_directory, '(자동차보험) 고객별 사고 발생률 예측 모델링_1.csv')
  output$dataset2 <- paste0(root_directory, '(자동차보험) 고객별 사고 발생률 예측 모델링_2.csv')
  output$dataset3 <- paste0(root_directory, '(자동차보험) 고객별 사고 발생률 예측 모델링_3.csv')

  return(output)
}


In [3]:
# resource_paths 변수에 데이터셋 경로 할당
resource_paths <- initialize_resource_paths()

In [7]:
read_user_enriched <- function(input) {
  output <- read.csv(input, sep = ',')
  return(output)
}


In [9]:
# 데이터셋 불러오기
dataset1 <- read_user_enriched(resource_paths[['dataset1']])
dataset2 <- read_user_enriched(resource_paths[['dataset2']])
dataset3 <- read_user_enriched(resource_paths[['dataset3']])

# 메모리 사용량 확인
memory_usage <- object.size(list(dataset1, dataset2, dataset3))
print(paste("RAM memory usage:", memory_usage, "bytes"))


Warning message in file(file, "rt"):
“cannot open file '/content/drive/MyDrive/Dataset/(자동차보험) 고객별 사고 발생률 예측 모델링_1.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
# 데이터셋 합치기
df1 <- dataset1[, 1:15]
df1 <- df1[complete.cases(df1), ]  # 결측치 제거
df <- rbind(df1, dataset2, dataset3)

# 문자열 형식 수정 및 열 추가
df$YUHO <- as.integer(gsub(',', '', df$YUHO))
df$SAGO <- as.integer(sapply(strsplit(as.character(gsub(',', '', df$SAGO)), "\\."), function(x) as.numeric(x[1])))
df$사고율 <- ifelse(df$YUHO == 0, 0, df$SAGO / df$YUHO)

# 변수 이름 변경
names(df) <- c('자동차', '연령대', '성별', '국산구분코드', '직전3년간사고건수', '차량경과년수', '배기량별차종', '운전자한정특별약관코드', '운전자한정특별약관', '가입경력코드', '차량가입금액', '영상기록장치특약', '마일리지특약', '유효대수', '사고건수', '사고유무')

# '사고유무' 열 생성
df$사고유무 <- ifelse(df$사고건수 == 0, 0, 1)

# 치환 매핑 적용
mapping <- c('N' = 0, 'D' = 1, 'C' = 2, 'B' = 3, 'Z' = '기타', '0' = '결측치')
df$직전3년간사고건수 <- mapping[df$직전3년간사고건수]

df
